# Notebook to demo how to maintain state in a session.

In [1]:
import os
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [2]:
import litellm
from gait import MAO, Agent

## Below is optional.

In [3]:
litellm.enable_cache()

## Define a tool as a class that holds a state.

Make sure to docstring the class, not the `__call__` function. 

In [4]:
class PercentageCalculator:
    """Calculate the percentage of a value.

    :param value: The value to calculate the percentage of.
    :param percentage: The percent value. Has to be between 0 and 100.
    :return: The percentage value.
    """

    def __init__(
        self,
        counter: int = 0,
    ) -> None:
        # Here we a re keeping an internal state as a counter.
        self._counter = counter

    def __call__(
        self,
        value: float,
        percentage: float,
    ) -> float:
        # Update the internal state.
        self._counter += 1
        # Perform the call.
        return value * percentage / 100.0

## Define another tool.

In [5]:
def subtract(
    a: float,
    b: float,
) -> float:
    """Subtract b from a.

    :param a: The value to subtract from.
    :param b: How much to subtract from a.
    :return: The residual.
    """
    return a - b

## Create an instance of the tool.

In [6]:
percent_calculator = PercentageCalculator()

## Create an agent with references to the tools.

In [9]:
mao = MAO(
    Agent(
        model=f"azure/{os.environ['AZURE_API_DEPLOYMENT']}",
        functions=[
            # Reference the instance.
            percent_calculator,
            # reference the function.
            subtract,
        ],
        temperature=0.0,
    )
)
for _ in mao("If I have 15 apples and give away 40%, how many are left?"):
    if _.content:
        mao.terminate()
        print(_.content)

If you have 15 apples and give away 40%, you will have 9 apples left.


## Display the internal state.

Note: call the above cell again, and see the state change :-)

In [10]:
percent_calculator._counter

2